In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
models = [['my_model_(drift_master).h5', 'my_model_(loosey-goosey).h5', 
          'my_model_(little_less_loosey-goosey).h5'], [0.4,0.6,0.6], [0.3,0.4,0.4]]

In [7]:
import pandas as pd

settings = pd.DataFrame(data=np.array(models).T, 
                        columns=['name', 'speed_limit', 'break_power'])

settings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         3 non-null      object
 1   speed_limit  3 non-null      object
 2   break_power  3 non-null      object
dtypes: object(3)
memory usage: 200.0+ bytes


In [4]:
model_number = 0 #choose your model

In [5]:
model_name = settings.iloc[model_number].loc['name']
speed_limit = float(settings.iloc[model_number].loc['speed_limit'])
break_power = float(settings.iloc[model_number].loc['break_power'])

In [6]:
type(speed_limit)

str

In [6]:
Actions = np.array([np.array([0, speed_limit, 0]),
np.array([0, 0, 0]),
np.array([0, 0, break_power]),
np.array([-1, 0, 0]),
np.array([1, 0, 0]),
np.array([-1, 0, break_power]),
np.array([1, 0, break_power])])

In [7]:
model = load_model(model_name)

In [8]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 96, 96, 32)        2432      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 48, 48, 64)        51264     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 24, 24, 64)        0  

In [9]:
import gym
env = gym.make('CarRacing-v0')
state = env.reset()
states = state.reshape(1,96,96,3)
# for i in range(5):
#     states[:,i] = state
    
for _ in range(50):
    action = np.array([0,1,0])
    next_state, reward, done, _ = env.step(action)
    env.render()
    states = next_state.reshape(1,96,96,3)
for _ in range(3000):
    action_dist = model.predict(states)
    action_num = np.argmax(action_dist)
    action = Actions[action_num]
    next_state, reward, done, _ = env.step(action)
    env.render()
    states = next_state.reshape(1,96,96,3)
    if done:
        break
env.close()

Track generation: 1066..1344 -> 278-tiles track


D:\Anaconda\envs\CarRace\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


TypeError: bad operand type for unary -: 'numpy.str_'